In [1]:
import os
import sys
sys.path.append('/mnt/c/Users/44yos/RacePrediction/resnet_win5')
from data_setup.race_data.this_week_race_url import this_week_race_url
from data_setup.race_data.this_week_race_html import this_week_race_html
from data_setup.race_data.make_csv_from_this_week_html import make_csv_from_this_week_html
# from data_setup.race_data.make_csv_this_week_with_odds import make_csv_this_week_with_odds
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import os
from os import path

from pickle import load

from models import resnet
from tensorflow.python import keras
from tensorflow.python.keras.models import load_model
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

In [2]:
this_week_race_url()

In [3]:
this_week_race_html()

In [4]:
os.makedirs('../data/csv/2022/May_1', exist_ok = True)
make_csv_from_this_week_html()

In [5]:
#make_csv_this_week_with_odds()

In [6]:
race_df = pd.read_csv("../data/csv/2022/May_1/5_1.csv", sep = ",")

In [7]:
print(len(race_df['race_id'].unique()))
print(race_df.shape)
print(race_df.tail(100))

6
(85, 22)
            race_id race_round race_title      race_distance weather  \
0   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
1   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
2   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
3   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
4   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
5   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
6   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
7   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
8   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
9   202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
10  202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
11  202203010610&rf        10R      喜多方特別       ダ1700m (右)\n       晴   
12  202203010610&rf        10R      喜多方特別       ダ1700

In [8]:
odds = pd.DataFrame(columns = ["odds"])
odds["odds"] = [2.8,5.7,37.1,38.3,12.3,8.7,26.0,36.9,0.0,32.6,10.7,17.6,50.4,5.0,84.5,25.9,72.9,4.6,7.0,26.1,17.6,13.5,10.0,4.4,51.1,4.9,24.8,33.7,21.2,104.3,21.2,35.0,32.7,32.8,49.1,6.6,9.4,10.7,108.8,20.8,84.2,7.0,4.6,18.3,9.3,6.3,49.1,7.9,46.1,3.3,5.0,18.2,4.6,5.0,13.8,23.9,32.1,3.8,32.1,6.4,4.7,47.3,31.6,3.3,13.2,12.3,20.0,7.9,25.1,54.7,71.0,12.0,87.3,9.2,89.2,11.0,103.4,63.4,45.2,39.3,102.3,35.4,5.2,29.7,2.4]

In [9]:
print(odds.shape)
# print(odds)

(85, 1)


In [10]:
race_df["odds"] = odds["odds"]

In [11]:
race_df.tail(50)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
35,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,3,6,2016105713,牡6,55.0,1096,?,?,?,6.6,?
36,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,4,7,2017101323,牡5,54.0,1118,?,?,?,9.4,?
37,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,4,8,2017104867,牡5,56.0,5212,?,?,?,10.7,?
38,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,5,9,2015105411,牡7,54.0,1192,?,?,?,108.8,?
39,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,5,10,2017100887,牝5,54.0,1019,?,?,?,20.8,?
40,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,6,11,2016102214,牡6,55.0,1147,?,?,?,84.2,?
41,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,6,12,2016100617,牡6,56.0,1170,?,?,?,7.0,?
42,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,7,13,2016105013,牡6,55.0,1179,?,?,?,4.6,?
43,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,7,14,2015102979,牡7,55.0,5386,?,?,?,18.3,?
44,202205020410&rf,10R,ブリリアントS,ダ2100m (左)\n,晴,良,14:50発走,2022-5-1,東京,16,?,8,15,2017102011,牡5,54.0,1117,?,?,?,9.3,?


race_round

In [12]:
race_df['race_round'] = race_df['race_round'].str.strip('R \n')

In [13]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight
0,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?
1,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?
2,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?
3,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?
4,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?


race_rank

In [14]:
race_df["race_title"] = race_df["race_title"].replace('天皇賞(春)','天皇賞(春)G1')
# race_df["race_title"] = race_df["race_title"].replace('京都牝馬S','京都牝馬SG3')
# race_df["race_title"] = race_df["race_title"].replace('阪神C','阪神CG2')

In [15]:
race_rank = race_df["race_title"].str.extract('(G1|G2|G3)', expand = True)
race_rank.columns = {"race_rank"}

race_df = pd.concat([race_df, race_rank], axis = 1)
race_df.fillna(value = {'race_rank': 0}, inplace = True) # 元もデータに反映
race_df['race_rank'] = race_df['race_rank'].replace('G1', 3)
race_df['race_rank'] = race_df['race_rank'].replace('G2', 2)
race_df['race_rank'] = race_df['race_rank'].replace('G3', 1)

In [16]:
print("race_rank:", race_df["race_rank"].value_counts())

race_rank: 0    67
3    18
Name: race_rank, dtype: int64


In [17]:
race_df.head(5)

,race_id,race_round,race_title,race_distance,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank
0,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0
1,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0
2,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0
3,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0
4,202203010610&rf,10,喜多方特別,ダ1700m (右)\n,晴,良,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0


race_distance

In [18]:
race_df["race_distance"].unique()

array([' ダ1700m (右)\n', ' ダ2100m (左)\n', ' 芝1800m (左)\n', ' ダ1400m (右)\n',
       ' 芝3200m (右 外-内)\n'], dtype=object)

In [19]:
ground_type = race_df["race_distance"].str.extract('(芝|ダ)', expand = True)
circle = race_df["race_distance"].str.extract('(右|左)', expand = True)
distance = race_df["race_distance"].str.extract('(\d+)m', expand = True)

ground_type.columns = {"ground_type"}
circle.columns = {"circle"}
distance.columns = {"distance"}

race_df = pd.concat([race_df, ground_type], axis = 1)
race_df = pd.concat([race_df, circle], axis = 1)
race_df = pd.concat([race_df, distance], axis = 1)

race_df = pd.get_dummies(race_df, columns = ["ground_type", "circle"])

In [20]:
race_df["distance"] = race_df["distance"].astype(int)
race_df.drop(['race_distance'], axis = 1, inplace = True)

In [21]:
# # 存在しない右左回り適宜追加 #########
add_circle = pd.DataFrame(columns = ["circle_左"])
race_df = pd.concat([race_df,add_circle], axis = 1).fillna(0)

In [22]:
race_df.head(5)

,race_id,race_round,race_title,weather,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左
0,202203010610&rf,10,喜多方特別,晴,良,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0
1,202203010610&rf,10,喜多方特別,晴,良,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0
2,202203010610&rf,10,喜多方特別,晴,良,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0
3,202203010610&rf,10,喜多方特別,晴,良,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0
4,202203010610&rf,10,喜多方特別,晴,良,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0


weather

In [23]:
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中山', '小雨')
# race_df["weather"] = race_df["weather"].mask(race_df["place"] == '中京', '晴')
#race_df["weather"] = race_df["weather"].mask(race_df["place"] == '札幌', '晴')

In [24]:
weather = race_df["weather"].str.extract('(晴|曇|小雨|雨|小雪|雪)', expand = True)
weather.columns = {"weather_circumstance"}
race_df = pd.concat([race_df, weather], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["weather_circumstance"])
race_df.drop(['weather'], axis = 1, inplace = True)

In [25]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴
0,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1
1,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1
2,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1
3,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1
4,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1


In [26]:
# 存在しない天気適宜追加 #########
add_weather = pd.DataFrame(columns = ["weather_circumstance_小雨","weather_circumstance_雨","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_小雪","weather_circumstance_雪"])
race_df = pd.concat([race_df,add_weather], axis = 1).fillna(0)

In [27]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
1,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
2,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
3,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
4,202203010610&rf,10,喜多方特別,良,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0


ground_condition

In [28]:
# race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '中山', '稍重')
#race_df["ground_condition"] = race_df["ground_condition"].mask(race_df["place"] == '新潟', '良')
#race_df["ground_condition"] = "良"

In [29]:
race_df["ground_condition"] = race_df["ground_condition"].replace('(不良)', 4, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(稍重)', 2, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(重)', 3, regex = True)
race_df["ground_condition"] = race_df["ground_condition"].replace('(良)', 1, regex = True)

In [30]:
print("ground_condition:", race_df["ground_condition"].value_counts())

ground_condition: 1    85
Name: ground_condition, dtype: int64


In [31]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,time,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202203010610&rf,10,喜多方特別,1,14:40発走,2022-5-1,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
1,202203010610&rf,10,喜多方特別,1,14:40発走,2022-5-1,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
2,202203010610&rf,10,喜多方特別,1,14:40発走,2022-5-1,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
3,202203010610&rf,10,喜多方特別,1,14:40発走,2022-5-1,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
4,202203010610&rf,10,喜多方特別,1,14:40発走,2022-5-1,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0


datetime

In [32]:
race_df["date"] =  pd.to_datetime(race_df['date'])
race_df.drop(["time"], axis = 1, inplace = True)

In [33]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,place,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪
0,202203010610&rf,10,喜多方特別,1,2022-05-01,福島,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
1,202203010610&rf,10,喜多方特別,1,2022-05-01,福島,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,福島,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
3,202203010610&rf,10,喜多方特別,1,2022-05-01,福島,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0
4,202203010610&rf,10,喜多方特別,1,2022-05-01,福島,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0


place

In [34]:
race_df = pd.get_dummies(race_df, columns = ["place"])

In [35]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
3,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0
4,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0


In [36]:
# 存在しない場所追加 '札幌', '中京', '新潟', '函館', '小倉', '京都','東京','福島'  '阪神' '中山'
add_place = pd.DataFrame(columns =['札幌', '中京', '新潟', '函館', '小倉', '京都','東京','福島','中山'])
race_df = pd.concat([race_df,add_place], axis = 1).fillna(0)

In [37]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,sex_and_age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,中山
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,牡5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,セ5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,牡4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,4,2017100275,牝5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
4,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,5,2017105170,牡5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


sex_age

In [38]:
sex = race_df["sex_and_age"].str.extract('(牡|牝|セ)', expand = True)
sex.columns = {"sex"}
race_df = pd.concat([race_df, sex], axis = 1)
race_df = pd.get_dummies(race_df, columns = ["sex"])

race_df["sex_and_age"] = race_df["sex_and_age"].str.strip("牡牝セ\n")
race_df["sex_and_age"] = race_df["sex_and_age"].astype(int)
race_df = race_df.rename(columns = {"sex_and_age": "age"})

In [39]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,中山,sex_セ,sex_牝,sex_牡
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
3,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,4,2017100275,5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
4,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,5,2017105170,5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


burden_weight

In [40]:
race_df["burden_weight"] = race_df["burden_weight"].astype(float)

odds

In [41]:
print(race_df["odds"].dtypes)

float64


In [42]:
race_df.to_csv("../data/csv/2022/May_1/refined_5_1_race.csv", index = False)

In [43]:
#race_df = pd.read_csv("../data/csv/2021/dec_4/refined_12_26_race.csv", sep = ",")

make_train_data

In [44]:
horse_df = pd.read_csv("../data/csv/horse_data/refined_horse_data.csv", sep = ",")
horse_f_df = pd.read_csv("../data/csv/horse_data/horse_featured_data.csv", sep = ",")

/home/yoshi/miniconda3/envs/tensorflow-gpu/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [45]:
race_df['order'] = race_df['order'].astype(str)
race_df['horse_id'] = race_df['horse_id'].astype(str)
race_df['jockey_id'] = race_df['jockey_id'].astype(str)
race_df['date'] = pd.to_datetime(race_df['date'])
print(race_df.shape)
print(race_df.dtypes)
race_df.head(3)

(85, 47)
race_id                            object
race_round                         object
race_title                         object
ground_condition                    int64
date                       datetime64[ns]
total_horse_number                  int64
order                              object
frame_number                        int64
horse_number                        int64
horse_id                           object
age                                 int64
burden_weight                     float64
jockey_id                          object
goal_time                          object
half_order                         object
last_time                          object
odds                              float64
horse_weight                       object
race_rank                           int64
distance                            int64
ground_type_ダ                       uint8
ground_type_芝                       uint8
circle_右                            uint8
circle_左                 

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,中山,sex_セ,sex_牝,sex_牡
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1


In [46]:
horse_df['order'] = horse_df['order'].astype(str)
horse_df['horse_id'] = horse_df['horse_id'].astype(str)
horse_df['date'] = pd.to_datetime(horse_df['date'])
print(horse_df.dtypes)
print(horse_df.shape)
horse_df.head(3)

horse_id                           object
father_id                          object
grandfather_id                     object
mother_id                          object
total_race_number                   int64
date                       datetime64[ns]
whole_horse_number                  int64
odds                              float64
order                              object
jockey_id                          object
burden_weight                     float64
race_distance                       int64
ground_condition                    int64
goal_time                         float64
half_order                        float64
last_time                         float64
horse_weight                      float64
weather_circumstance_小雨             int64
weather_circumstance_小雪             int64
weather_circumstance_晴              int64
weather_circumstance_曇              int64
weather_circumstance_雨              int64
weather_circumstance_雪              int64
main_place_その他                    

,horse_id,father_id,grandfather_id,mother_id,total_race_number,date,whole_horse_number,odds,order,jockey_id,burden_weight,race_distance,ground_condition,goal_time,half_order,last_time,horse_weight,weather_circumstance_小雨,weather_circumstance_小雪,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_雨,weather_circumstance_雪,main_place_その他,main_place_中京,main_place_中山,main_place_京都,main_place_函館,main_place_小倉,main_place_新潟,main_place_札幌,main_place_東京,main_place_福島,main_place_阪神,race_rank,ground_type_ダ,ground_type_芝,ground_type_障,horse_weight_dif
0,2018100039,2012104511,2001103460,2009110042,7,2021-11-27,16,76.4,1,1150,56.0,1400,2,85.1,0.062500,36.0,486.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,-6.0
1,2018100039,2012104511,2001103460,2009110042,7,2021-10-10,15,14.6,11,1091,53.0,1200,1,71.3,0.166667,37.9,492.0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,4.0
2,2018100039,2012104511,2001103460,2009110042,7,2021-09-18,14,17.3,1,1178,54.0,1200,4,70.8,0.250000,36.6,488.0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,-4.0


In [47]:
horse_f_df['horse_id'] = horse_f_df['horse_id'].astype(str)
print(horse_f_df.dtypes)
print(horse_f_df.shape)
horse_f_df.head(3)

horse_id           object
grass_win_rate    float64
dart_win_rate     float64
win_rate          float64
dtype: object
(61317, 4)


,horse_id,grass_win_rate,dart_win_rate,win_rate
0,2018100039,0.0,0.600000,0.428571
1,2018100040,0.3,0.000000,0.300000
2,2018100047,0.0,0.666667,0.500000


In [48]:
pedigree_map = pd.read_csv("../data/csv/horse_data/pedigree_map.csv", sep = ",")
pedigree_map['horse_id'] = pedigree_map['horse_id'].astype(str)
print(pedigree_map.dtypes)

horse_id               object
f_grass_win_rate      float64
f_dart_win_rate       float64
f_win_rate            float64
g_f_grass_win_rate    float64
g_f_dart_win_rate     float64
g_f_win_rate          float64
m_grass_win_rate      float64
m_dart_win_rate       float64
m_win_rate            float64
dtype: object


add_pedigree_map to race_df

In [49]:
race_df = pd.merge(race_df, pedigree_map, on = "horse_id", how = "left").fillna(0)

In [50]:
race_df.head(5)

,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,jockey_id,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,中山,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,g_f_win_rate,m_grass_win_rate,m_dart_win_rate,m_win_rate
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,5,57.0,1130,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.111111,0.416667,0.285714
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,5,57.0,1157,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.000000,0.000000
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,4,57.0,1138,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.333333,0.000000,0.318182,0.320,0.5,0.333333,0.000000,0.052632,0.050000
3,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,4,2017100275,5,55.0,726,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.200000,0.571429,0.352941,0.875,0.0,0.875000,0.000000,0.263158,0.238095
4,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,5,2017105170,5,57.0,1127,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.388889,0.000000,0.388889,0.875,0.0,0.875000,0.214286,0.363636,0.280000


add previous race

In [51]:
#delete unnecessary columns in horse_df
horse_df.drop(["father_id", "grandfather_id", "mother_id", "total_race_number"], axis = 1, inplace = True)

In [52]:
tmp_date_race_data = race_df[["date", "horse_id", "jockey_id"]]

In [53]:
#function 
def make_one_horse_data(tmp_date_horse_data, one_horse_df):
    one_horse_df = one_horse_df.sort_values('date', ascending = False) #False = 近いやつが上
    one_horse_df["same_jockey"] = (tmp_date_horse_data["jockey_id"] == one_horse_df["jockey_id"]) * 1.0
   
    
    one_horse_df = one_horse_df[(one_horse_df["date"] < tmp_date_horse_data["date"])] #レース前を抽出
    
    
    if (len(one_horse_df) <= 2):
        n = len(one_horse_df) 
    else:
        n = 3
    
    #print("one_horse_df")
    #print(one_horse_df)
    
    result_df = one_horse_df.copy()
    result_df.columns = result_df.columns + "_1"
    #print("result_df")
    #print(result_df)
    for i in range(1,n):
        shift_df = one_horse_df.shift(-i)
        #print("shift_df")
        #print(shift_df)
        shift_df.drop(["date"], axis = 1, inplace = True)
        shift_df.drop(["jockey_id"], axis = 1, inplace = True)
        shift_df.columns = shift_df.columns + "_" + str(i+1)
        result_df = pd.concat([result_df, shift_df], axis = 1)
        #print("result_df")
        #print(result_df)
    
    
    return result_df.head(1)                  #result_df.dropna()
        

In [54]:
one_data = tmp_date_race_data.iloc[0].copy()
one_horse_data = horse_df[horse_df["horse_id"] == "2018106606"].copy()
#print(one_data)
#print(one_horse_data)
tmp_race_df = make_one_horse_data(one_data, one_horse_data)
#print(tmp_race_df)

In [55]:
# うまくいくけど時間かかるやつ

for index, tmp in race_df.iterrows():
    tmp_race_df = tmp_race_df.append(make_one_horse_data(tmp, horse_df[horse_df["horse_id"] == tmp["horse_id"]]))


In [56]:
print(tmp_race_df.head(10))

       horse_id_1     date_1  whole_horse_number_1  odds_1 order_1  \
13763  2018106606 2021-11-21                    16    16.6       4   
35146  2017100468 2021-11-14                     9     7.4       6   
38180  2017103688 2021-11-20                     8    10.2       2   
9098   2018103473 2021-08-22                    14     6.8       7   
14673  2017100275 2021-11-13                    16    32.4       1   
23351  2017105170 2021-10-31                    16    21.7       8   
6385   2018100821 2021-12-11                    16    16.4      16   
8772   2018103256 2021-11-28                    16    88.0      12   
49205  2016102372 2021-11-21                    14   169.4       9   
26226  2017106662 2021-10-23                    14    78.1      14   

      jockey_id_1  burden_weight_1  race_distance_1  ground_condition_1  \
13763        1135             53.0             2400                   1   
35146        1180             57.0             1800                   1   
3818

In [57]:
# tmp_race_df.to_pickle("../data/csv/2021/tmp_race_12_26_df.pkl")
#tmp_race_df = pd.read_pickle("../data/csv/2021/tmp_race_12_25_df.pkl")

In [58]:
#delete horse1,2,3_id
tmp_race_df.drop(['horse_id_2','horse_id_3'], axis = 1, inplace = True)

In [59]:
#tmp_race_df = tmp_race_df.dropna()
tmp_race_df["whole_horse_number_2"] = tmp_race_df["whole_horse_number_2"].fillna(tmp_race_df["whole_horse_number_1"])
tmp_race_df["odds_2"] = tmp_race_df["odds_2"].fillna(tmp_race_df["odds_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["burden_weight_2"] = tmp_race_df["burden_weight_2"].fillna(tmp_race_df["burden_weight_1"])
tmp_race_df["race_distance_2"] = tmp_race_df["race_distance_2"].fillna(tmp_race_df["race_distance_1"])
tmp_race_df["ground_condition_2"] = tmp_race_df["ground_condition_2"].fillna(tmp_race_df["ground_condition_1"])
tmp_race_df["order_2"] = tmp_race_df["order_2"].fillna(tmp_race_df["order_1"])
tmp_race_df["goal_time_2"] = tmp_race_df["goal_time_2"].fillna(tmp_race_df["goal_time_1"])
#tmp_race_df["velocity_2"] = tmp_race_df["velocity_2"].fillna(tmp_race_df["velocity_1"])
tmp_race_df["half_order_2"] = tmp_race_df["half_order_2"].fillna(tmp_race_df["half_order_1"])
tmp_race_df["last_time_2"] = tmp_race_df["last_time_2"].fillna(tmp_race_df["last_time_1"])
tmp_race_df["horse_weight_2"] = tmp_race_df["horse_weight_2"].fillna(tmp_race_df["horse_weight_1"])
tmp_race_df["weather_circumstance_小雨_2"] = tmp_race_df["weather_circumstance_小雨_2"].fillna(tmp_race_df["weather_circumstance_小雨_1"])
tmp_race_df["weather_circumstance_小雪_2"] = tmp_race_df["weather_circumstance_小雪_2"].fillna(tmp_race_df["weather_circumstance_小雪_1"])
tmp_race_df["weather_circumstance_晴_2"] = tmp_race_df["weather_circumstance_晴_2"].fillna(tmp_race_df["weather_circumstance_晴_1"])
tmp_race_df["weather_circumstance_曇_2"] = tmp_race_df["weather_circumstance_曇_2"].fillna(tmp_race_df["weather_circumstance_曇_1"])
tmp_race_df["weather_circumstance_雨_2"] = tmp_race_df["weather_circumstance_雨_2"].fillna(tmp_race_df["weather_circumstance_雨_1"])
tmp_race_df["weather_circumstance_雪_2"] = tmp_race_df["weather_circumstance_雪_2"].fillna(tmp_race_df["weather_circumstance_雪_1"])
tmp_race_df["main_place_その他_2"] = tmp_race_df["main_place_その他_2"].fillna(tmp_race_df["main_place_その他_1"])
tmp_race_df["main_place_中京_2"] = tmp_race_df["main_place_中京_2"].fillna(tmp_race_df["main_place_中京_1"])
tmp_race_df["main_place_中山_2"] = tmp_race_df["main_place_中山_2"].fillna(tmp_race_df["main_place_中山_1"])
tmp_race_df["main_place_京都_2"] = tmp_race_df["main_place_京都_2"].fillna(tmp_race_df["main_place_京都_1"])
tmp_race_df["main_place_函館_2"] = tmp_race_df["main_place_函館_2"].fillna(tmp_race_df["main_place_函館_1"])
tmp_race_df["main_place_小倉_2"] = tmp_race_df["main_place_小倉_2"].fillna(tmp_race_df["main_place_小倉_1"])
tmp_race_df["main_place_新潟_2"] = tmp_race_df["main_place_新潟_2"].fillna(tmp_race_df["main_place_新潟_1"])
tmp_race_df["main_place_札幌_2"] = tmp_race_df["main_place_札幌_2"].fillna(tmp_race_df["main_place_札幌_1"])
tmp_race_df["main_place_東京_2"] = tmp_race_df["main_place_東京_2"].fillna(tmp_race_df["main_place_東京_1"])
tmp_race_df["main_place_福島_2"] = tmp_race_df["main_place_福島_2"].fillna(tmp_race_df["main_place_福島_1"])
tmp_race_df["main_place_阪神_2"] = tmp_race_df["main_place_阪神_2"].fillna(tmp_race_df["main_place_阪神_1"])
tmp_race_df["race_rank_2"] = tmp_race_df["race_rank_2"].fillna(tmp_race_df["race_rank_1"])
tmp_race_df["ground_type_ダ_2"] = tmp_race_df["ground_type_ダ_2"].fillna(tmp_race_df["ground_type_ダ_1"])
tmp_race_df["ground_type_芝_2"] = tmp_race_df["ground_type_芝_2"].fillna(tmp_race_df["ground_type_芝_1"])
tmp_race_df["ground_type_障_2"] = tmp_race_df["ground_type_障_2"].fillna(tmp_race_df["ground_type_障_1"])
tmp_race_df["horse_weight_dif_2"] = tmp_race_df["horse_weight_dif_2"].fillna(tmp_race_df["horse_weight_dif_1"])
tmp_race_df["same_jockey_2"] = tmp_race_df["same_jockey_2"].fillna(tmp_race_df["same_jockey_1"])

tmp_race_df["whole_horse_number_3"] = tmp_race_df["whole_horse_number_3"].fillna(tmp_race_df["whole_horse_number_2"])
tmp_race_df["odds_3"] = tmp_race_df["odds_3"].fillna(tmp_race_df["odds_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["burden_weight_3"] = tmp_race_df["burden_weight_3"].fillna(tmp_race_df["burden_weight_2"])
tmp_race_df["race_distance_3"] = tmp_race_df["race_distance_3"].fillna(tmp_race_df["race_distance_2"])
tmp_race_df["ground_condition_3"] = tmp_race_df["ground_condition_3"].fillna(tmp_race_df["ground_condition_2"])
tmp_race_df["order_3"] = tmp_race_df["order_3"].fillna(tmp_race_df["order_2"])
tmp_race_df["goal_time_3"] = tmp_race_df["goal_time_3"].fillna(tmp_race_df["goal_time_2"])
#tmp_race_df["velocity_3"] = tmp_race_df["velocity_3"].fillna(tmp_race_df["velocity_2"])
tmp_race_df["half_order_3"] = tmp_race_df["half_order_3"].fillna(tmp_race_df["half_order_2"])
tmp_race_df["last_time_3"] = tmp_race_df["last_time_3"].fillna(tmp_race_df["last_time_2"])
tmp_race_df["horse_weight_3"] = tmp_race_df["horse_weight_3"].fillna(tmp_race_df["horse_weight_2"])
tmp_race_df["weather_circumstance_小雨_3"] = tmp_race_df["weather_circumstance_小雨_3"].fillna(tmp_race_df["weather_circumstance_小雨_2"])
tmp_race_df["weather_circumstance_小雪_3"] = tmp_race_df["weather_circumstance_小雪_3"].fillna(tmp_race_df["weather_circumstance_小雪_2"])
tmp_race_df["weather_circumstance_晴_3"] = tmp_race_df["weather_circumstance_晴_3"].fillna(tmp_race_df["weather_circumstance_晴_2"])
tmp_race_df["weather_circumstance_曇_3"] = tmp_race_df["weather_circumstance_曇_3"].fillna(tmp_race_df["weather_circumstance_曇_2"])
tmp_race_df["weather_circumstance_雨_3"] = tmp_race_df["weather_circumstance_雨_3"].fillna(tmp_race_df["weather_circumstance_雨_2"])
tmp_race_df["weather_circumstance_雪_3"] = tmp_race_df["weather_circumstance_雪_3"].fillna(tmp_race_df["weather_circumstance_雪_2"])
tmp_race_df["main_place_その他_3"] = tmp_race_df["main_place_その他_3"].fillna(tmp_race_df["main_place_その他_2"])
tmp_race_df["main_place_中京_3"] = tmp_race_df["main_place_中京_3"].fillna(tmp_race_df["main_place_中京_2"])
tmp_race_df["main_place_中山_3"] = tmp_race_df["main_place_中山_3"].fillna(tmp_race_df["main_place_中山_2"])
tmp_race_df["main_place_京都_3"] = tmp_race_df["main_place_京都_3"].fillna(tmp_race_df["main_place_京都_2"])
tmp_race_df["main_place_函館_3"] = tmp_race_df["main_place_函館_3"].fillna(tmp_race_df["main_place_函館_2"])
tmp_race_df["main_place_小倉_3"] = tmp_race_df["main_place_小倉_3"].fillna(tmp_race_df["main_place_小倉_2"])
tmp_race_df["main_place_新潟_3"] = tmp_race_df["main_place_新潟_3"].fillna(tmp_race_df["main_place_新潟_2"])
tmp_race_df["main_place_札幌_3"] = tmp_race_df["main_place_札幌_3"].fillna(tmp_race_df["main_place_札幌_2"])
tmp_race_df["main_place_東京_3"] = tmp_race_df["main_place_東京_3"].fillna(tmp_race_df["main_place_東京_2"])
tmp_race_df["main_place_福島_3"] = tmp_race_df["main_place_福島_3"].fillna(tmp_race_df["main_place_福島_2"])
tmp_race_df["main_place_阪神_3"] = tmp_race_df["main_place_阪神_3"].fillna(tmp_race_df["main_place_阪神_2"])
tmp_race_df["race_rank_3"] = tmp_race_df["race_rank_3"].fillna(tmp_race_df["race_rank_2"])
tmp_race_df["ground_type_ダ_3"] = tmp_race_df["ground_type_ダ_3"].fillna(tmp_race_df["ground_type_ダ_2"])
tmp_race_df["ground_type_芝_3"] = tmp_race_df["ground_type_芝_3"].fillna(tmp_race_df["ground_type_芝_2"])
tmp_race_df["ground_type_障_3"] = tmp_race_df["ground_type_障_3"].fillna(tmp_race_df["ground_type_障_2"])
tmp_race_df["horse_weight_dif_3"] = tmp_race_df["horse_weight_dif_3"].fillna(tmp_race_df["horse_weight_dif_2"])
tmp_race_df["same_jockey_3"] = tmp_race_df["same_jockey_3"].fillna(tmp_race_df["same_jockey_2"])



In [60]:
print(tmp_race_df.shape)
tmp_race_df.isnull().sum()

(77, 102)


horse_id_1                   0
date_1                       0
whole_horse_number_1         0
odds_1                       0
order_1                      0
jockey_id_1                  0
burden_weight_1              0
race_distance_1              0
ground_condition_1           0
goal_time_1                  0
half_order_1                 0
last_time_1                  0
horse_weight_1               0
weather_circumstance_小雨_1    0
weather_circumstance_小雪_1    0
weather_circumstance_晴_1     0
weather_circumstance_曇_1     0
weather_circumstance_雨_1     0
weather_circumstance_雪_1     0
main_place_その他_1             0
main_place_中京_1              0
main_place_中山_1              0
main_place_京都_1              0
main_place_函館_1              0
main_place_小倉_1              0
main_place_新潟_1              0
main_place_札幌_1              0
main_place_東京_1              0
main_place_福島_1              0
main_place_阪神_1              0
race_rank_1                  0
ground_type_ダ_1              0
ground_t

In [61]:
race_df = pd.merge(race_df, tmp_race_df, left_on = ["horse_id"], right_on = ["horse_id_1"], how = "left")

same_jockey

In [62]:
same_jockey = pd.DataFrame(columns = ["same_jockey"])
same_jockey["same_jockey"] = (race_df["jockey_id_1"] == race_df["jockey_id"]) * 1.0 

race_df = pd.concat([race_df,same_jockey], axis = 1)
race_df.drop(["jockey_id"], axis = 1,inplace = True)

In [63]:
race_df.drop(['horse_id_1','date_1'], axis = 1, inplace = True)

In [64]:
race_df["horse_id"] = race_df["horse_id"].drop_duplicates()
race_df = race_df.dropna(subset = {"horse_id"})

In [65]:
print(race_df.shape)
race_df.head(5)

(85, 156)


,race_id,race_round,race_title,ground_condition,date,total_horse_number,order,frame_number,horse_number,horse_id,age,burden_weight,goal_time,half_order,last_time,odds,horse_weight,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_東京,place_福島,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,中山,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,...,main_place_中京_2,main_place_中山_2,main_place_京都_2,main_place_函館_2,main_place_小倉_2,main_place_新潟_2,main_place_札幌_2,main_place_東京_2,main_place_福島_2,main_place_阪神_2,race_rank_2,ground_type_ダ_2,ground_type_芝_2,ground_type_障_2,horse_weight_dif_2,same_jockey_2,whole_horse_number_3,odds_3,order_3,burden_weight_3,race_distance_3,ground_condition_3,goal_time_3,half_order_3,last_time_3,horse_weight_3,weather_circumstance_小雨_3,weather_circumstance_小雪_3,weather_circumstance_晴_3,weather_circumstance_曇_3,weather_circumstance_雨_3,weather_circumstance_雪_3,main_place_その他_3,main_place_中京_3,main_place_中山_3,main_place_京都_3,main_place_函館_3,main_place_小倉_3,main_place_新潟_3,main_place_札幌_3,main_place_東京_3,main_place_福島_3,main_place_阪神_3,race_rank_3,ground_type_ダ_3,ground_type_芝_3,ground_type_障_3,horse_weight_dif_3,same_jockey_3,same_jockey
0,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,1,1,2017100468,5,57.0,?,?,?,2.8,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.000000,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,10.0,1.1,1,56.0,1400.0,1.0,89.2,0.100000,38.5,510.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,7.0,0.0,0.0
1,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,2,2017103688,5,57.0,?,?,?,5.7,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0.000000,0.000000,0.000000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,14.0,0.0,11.0,21.1,8,57.0,1800.0,3.0,111.8,0.500000,39.6,468.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,0.0
2,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,2,3,2018103473,4,57.0,?,?,?,37.1,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.333333,0.000000,0.318182,0.320,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,13.0,11.8,1,56.0,1700.0,1.0,105.9,0.192308,37.2,426.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,0.0,0.0
3,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,4,2017100275,5,55.0,?,?,?,38.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0.200000,0.571429,0.352941,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1.0,15.0,21.4,4,55.0,1700.0,2.0,106.5,0.633333,38.4,450.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,-4.0,1.0,0.0
4,202203010610&rf,10,喜多方特別,1,2022-05-01,15,?,3,5,2017105170,5,57.0,?,?,?,12.3,?,0,1700,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.388889,0.000000,0.388889,0.875,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0,13.3,4,57.0,1400.0,3.0,83.6,0.812500,35.6,536.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,1.0,1.0


In [67]:
race_df.to_csv("../data/csv/2022/May_1/5_1_data.csv", index = False)

simple_keras

In [69]:
data = pd.read_csv("../data/csv/2021/dec_4/12_26_data.csv", sep = ",")

In [70]:
data['race_id'] = data['race_id'].astype(str)
data['race_round'] = data['race_round'].astype(str)
#data['total_horse_number'] = data['total_horse_number'].astype(str)
data['order'] = data['order'].astype(str)
data['frame_number'] = data['frame_number'].astype(str)
data['horse_number'] = data['horse_number'].astype(str)

In [71]:
print(data.head(5))

           race_id race_round race_title  ground_condition        date  \
0  202106050810&rf         10    フェアウェルS                 1  2021-12-26   
1  202106050810&rf         10    フェアウェルS                 1  2021-12-26   
2  202106050810&rf         10    フェアウェルS                 1  2021-12-26   
3  202106050810&rf         10    フェアウェルS                 1  2021-12-26   
4  202106050810&rf         10    フェアウェルS                 1  2021-12-26   

   total_horse_number order frame_number horse_number    horse_id  age  \
0                  16     ?            1            1  2016104491    5   
1                  16     ?            1            2  2017105389    4   
2                  16     ?            2            3  2017103616    4   
3                  16     ?            2            4  2017102588    4   
4                  16     ?            3            5  2016110131    5   

   burden_weight goal_time half_order last_time  odds horse_weight  race_rank  \
0           57.0         ?   

In [72]:
print(data.isnull().sum())

race_id                      0
race_round                   0
race_title                   0
ground_condition             0
date                         0
total_horse_number           0
order                        0
frame_number                 0
horse_number                 0
horse_id                     0
age                          0
burden_weight                0
goal_time                    0
half_order                   0
last_time                    0
odds                         0
horse_weight                 0
race_rank                    0
distance                     0
ground_type_ダ                0
ground_type_芝                0
circle_右                     0
circle_左                     0
weather_circumstance_晴       0
weather_circumstance_小雨      0
weather_circumstance_雨       0
weather_circumstance_晴.1     0
weather_circumstance_曇       0
weather_circumstance_小雪      0
weather_circumstance_雪       0
place_中山                     0
place_阪神                     0
札幌      

In [73]:
data = data.fillna(data.mean())

In [74]:
labels = data[["race_id","race_title", "order", "horse_number", "goal_time", "half_order", "last_time", "horse_weight"]]

In [75]:
row_x = data.drop(["race_id","race_round","race_title","date", "frame_number","order","horse_number","horse_id", "goal_time", "half_order","last_time", "horse_weight", ], axis = 1)
row_x.head(5)

,ground_condition,total_horse_number,age,burden_weight,odds,race_rank,distance,ground_type_ダ,ground_type_芝,circle_右,circle_左,weather_circumstance_晴,weather_circumstance_小雨,weather_circumstance_雨,weather_circumstance_晴.1,weather_circumstance_曇,weather_circumstance_小雪,weather_circumstance_雪,place_中山,place_阪神,札幌,中京,新潟,函館,小倉,京都,東京,福島,sex_セ,sex_牝,sex_牡,f_grass_win_rate,f_dart_win_rate,f_win_rate,g_f_grass_win_rate,g_f_dart_win_rate,g_f_win_rate,m_grass_win_rate,m_dart_win_rate,m_win_rate,whole_horse_number_1,odds_1,order_1,jockey_id_1,burden_weight_1,race_distance_1,ground_condition_1,goal_time_1,half_order_1,last_time_1,...,main_place_中京_2,main_place_中山_2,main_place_京都_2,main_place_函館_2,main_place_小倉_2,main_place_新潟_2,main_place_札幌_2,main_place_東京_2,main_place_福島_2,main_place_阪神_2,race_rank_2,ground_type_ダ_2,ground_type_芝_2,ground_type_障_2,horse_weight_dif_2,same_jockey_2,whole_horse_number_3,odds_3,order_3,burden_weight_3,race_distance_3,ground_condition_3,goal_time_3,half_order_3,last_time_3,horse_weight_3,weather_circumstance_小雨_3,weather_circumstance_小雪_3,weather_circumstance_晴_3,weather_circumstance_曇_3,weather_circumstance_雨_3,weather_circumstance_雪_3,main_place_その他_3,main_place_中京_3,main_place_中山_3,main_place_京都_3,main_place_函館_3,main_place_小倉_3,main_place_新潟_3,main_place_札幌_3,main_place_東京_3,main_place_福島_3,main_place_阪神_3,race_rank_3,ground_type_ダ_3,ground_type_芝_3,ground_type_障_3,horse_weight_dif_3,same_jockey_3,same_jockey
0,1,16,5,57.0,29.2,0,1200,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0.28,0.000000,0.280000,0.000000,0.0,0.000000,0.333333,0.0,0.333333,18,89.4,17,1181,53.0,1400,1,81.4,0.50000,35.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0,0.0,14.0,7.2,7,57.0,1200.0,2.0,71.1,0.571429,36.5,480.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,16,4,57.0,8.8,0,1200,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.50,0.769231,0.733333,0.142857,0.8,0.416667,0.000000,0.0,0.000000,16,7.2,13,1141,57.0,1150,1,68.9,0.43750,37.8,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-6.0,1.0,15.0,12.5,1,57.0,1200.0,1.0,70.8,0.133333,36.6,488.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-4.0,1.0,0.0
2,1,16,4,55.0,7.5,0,1200,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0.00,0.440000,0.392857,0.000000,0.0,0.000000,0.000000,0.0,0.000000,16,8.7,2,1091,55.0,1150,1,68.0,0.12500,37.3,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,12.0,9.1,2,55.0,1000.0,2.0,58.4,0.375000,34.9,464.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,4.0,1.0,1.0
3,1,16,4,57.0,14.9,0,1200,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,15,27.9,3,1030,54.0,1200,1,71.6,0.70000,35.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,6.0,0.0,15.0,19.5,9,57.0,1200.0,2.0,71.0,0.566667,36.2,510.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,1,16,5,57.0,7.1,0,1200,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,16,3.3,3,666,57.0,1200,1,71.9,0.78125,35.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,12.0,0.0,15.0,4.0,5,57.0,1200.0,1.0,71.1,1.000000,35.4,500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-2.0,0.0,1.0


In [76]:
#row_x = row_x.drop(["weather_circumstance_雪.1","weather_circumstance_小雪.1","weather_circumstance_雨.1","weather_circumstance_小雨.1","weather_circumstance_曇.1","weather_circumstance_晴.1"], axis = 1)
#row_x.tail(100)

In [77]:
# 列の順序変更   "小倉":'place_小倉',"阪神":'place_阪神',"東京":'place_東京',
row_x = row_x.rename(columns ={"東京":'place_東京',"小倉":'place_小倉', "中山":'place_中山',"中京":'place_中京',"札幌":'place_札幌',"函館":'place_函館',"福島":'place_福島',"京都":'place_京都',"新潟":'place_新潟'})
row_x = row_x[["ground_condition","total_horse_number","age","burden_weight","odds","race_rank","distance","ground_type_ダ","ground_type_芝","circle_右","circle_左","weather_circumstance_小雨","weather_circumstance_小雪","weather_circumstance_晴","weather_circumstance_曇","weather_circumstance_雨","weather_circumstance_雪","place_中京","place_中山","place_京都","place_函館","place_小倉","place_新潟","place_札幌","place_東京","place_福島","place_阪神","sex_セ","sex_牝","sex_牡","f_grass_win_rate","f_dart_win_rate","f_win_rate","g_f_grass_win_rate","g_f_dart_win_rate","g_f_win_rate","m_grass_win_rate","m_dart_win_rate","m_win_rate","same_jockey","whole_horse_number_1","odds_1","order_1","burden_weight_1","race_distance_1","ground_condition_1","goal_time_1","half_order_1","last_time_1","horse_weight_1","weather_circumstance_小雨_1","weather_circumstance_小雪_1","weather_circumstance_晴_1","weather_circumstance_曇_1","weather_circumstance_雨_1","weather_circumstance_雪_1","main_place_その他_1","main_place_中京_1","main_place_中山_1","main_place_京都_1","main_place_函館_1","main_place_小倉_1","main_place_新潟_1","main_place_札幌_1","main_place_東京_1","main_place_福島_1","main_place_阪神_1","race_rank_1","ground_type_ダ_1","ground_type_芝_1","ground_type_障_1","horse_weight_dif_1","same_jockey_1","whole_horse_number_2","odds_2","order_2","burden_weight_2","race_distance_2","ground_condition_2","goal_time_2","half_order_2","last_time_2","horse_weight_2","weather_circumstance_小雨_2","weather_circumstance_小雪_2","weather_circumstance_晴_2","weather_circumstance_曇_2","weather_circumstance_雨_2","weather_circumstance_雪_2","main_place_その他_2","main_place_中京_2","main_place_中山_2","main_place_京都_2","main_place_函館_2","main_place_小倉_2","main_place_新潟_2","main_place_札幌_2","main_place_東京_2","main_place_福島_2","main_place_阪神_2","race_rank_2","ground_type_ダ_2","ground_type_芝_2","ground_type_障_2","horse_weight_dif_2","same_jockey_2","whole_horse_number_3","odds_3","order_3","burden_weight_3","race_distance_3","ground_condition_3","goal_time_3","half_order_3","last_time_3","horse_weight_3","weather_circumstance_小雨_3","weather_circumstance_小雪_3","weather_circumstance_晴_3","weather_circumstance_曇_3","weather_circumstance_雨_3","weather_circumstance_雪_3","main_place_その他_3","main_place_中京_3","main_place_中山_3","main_place_京都_3","main_place_函館_3","main_place_小倉_3","main_place_新潟_3","main_place_札幌_3","main_place_東京_3","main_place_福島_3" ,"main_place_阪神_3","race_rank_3","ground_type_ダ_3","ground_type_芝_3","ground_type_障_3","horse_weight_dif_3","same_jockey_3"]]

In [78]:
# standard_scale = StandardScaler()
standard_scale = load(open("../training/standard_scale.pkl", "rb"))
x = standard_scale.fit_transform(row_x)
print(x.shape)

(58, 139)


In [79]:
# OWN_FILE_NAME = path.splitext(path.basename('\\Users\\vmlab\\win5.ext'))[0]

# week_model = load_model("../models/results/test.h5")

# week_model.summary()
# pred = week_model.predict(x)
# pred_order = np.argmax(pred, axis = 1)
# print(pred_order.shape)

In [91]:
new_model = resnet.ResRace(x.shape[1], 19)
new_model.load_weights("../models/results/test3.h5")
new_model.summary()
pred = new_model.predict(x)
pred_order = np.argmax(pred, axis = 1)

Model: "res_net_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_82 (Dense)             multiple                  13900     
_________________________________________________________________
bottleneck_24 (Bottleneck)   multiple                  31500     
_________________________________________________________________
bottleneck_25 (Bottleneck)   multiple                  31500     
_________________________________________________________________
dense_89 (Dense)             multiple                  20000     
_________________________________________________________________
bottleneck_26 (Bottleneck)   multiple                  123000    
_________________________________________________________________
bottleneck_27 (Bottleneck)   multiple                  123000    
_________________________________________________________________
dense_96 (Dense)             multiple                  80

In [92]:
pred_df = pd.DataFrame(pred, columns = ['0','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18'])
pred_order_df = pd.DataFrame(pred_order, columns = ["pred"])
summary = labels.join(pred_order_df)
summary = summary.join(row_x["odds"])

In [93]:
#pred_df = pred_df.drop(["0"], axis = 1)
summary = pd.concat([summary,pred_df], axis = 1)
print(pred_df)

               0         1         2         3         4         5         6  \
0   9.350047e-11  0.036098  0.053687  0.049254  0.068935  0.064917  0.076388   
1   8.788118e-12  0.177101  0.138714  0.085346  0.086248  0.066353  0.055803   
2   9.220748e-14  0.425450  0.221525  0.097887  0.065439  0.047000  0.027191   
3   2.461787e-11  0.013837  0.021294  0.021197  0.042139  0.040174  0.055881   
4   6.387919e-13  0.262855  0.203402  0.113614  0.093566  0.072765  0.051339   
5   1.469444e-11  0.010309  0.008949  0.010105  0.029395  0.015219  0.020587   
6   1.223644e-11  0.011083  0.013427  0.013043  0.030063  0.025509  0.035972   
7   2.648119e-11  0.018449  0.021382  0.019825  0.038672  0.032541  0.042483   
8   9.286166e-12  0.004981  0.004645  0.005839  0.019576  0.011552  0.017434   
9   5.443081e-12  0.004542  0.004606  0.005278  0.017532  0.011917  0.018412   
10  1.331937e-12  0.278719  0.179959  0.094859  0.079903  0.060483  0.043689   
11  3.657129e-12  0.009045  0.018114  0.

In [94]:
print(summary[["race_title","odds","horse_number", "pred"]])

   race_title   odds horse_number  pred
0     フェアウェルS   29.2            1     8
1     フェアウェルS    8.8            2     1
2     フェアウェルS    7.5            3     1
3     フェアウェルS   14.9            4     9
4     フェアウェルS    7.1            5     1
5     フェアウェルS   24.8            6    15
6     フェアウェルS   48.3            7    14
7     フェアウェルS   22.2            8    14
8     フェアウェルS   52.2            9    15
9     フェアウェルS   71.5           10    15
10    フェアウェルS    5.6           11     1
11    フェアウェルS   14.1           12     9
12    フェアウェルS   90.3           13     9
13    フェアウェルS   11.9           14     2
14    フェアウェルS    3.8           15     1
15    フェアウェルS   43.8           16    15
16     有馬記念G1   45.5            1    14
17     有馬記念G1   33.3            2     2
18     有馬記念G1   93.3            3    14
19     有馬記念G1   82.8            4    14
20     有馬記念G1   18.8            5     8
21     有馬記念G1   50.4            6    14
22     有馬記念G1    3.2            7     1
23     有馬記念G1   99.4            8    14


In [95]:
# summary[["race_title","odds","horse_number","pred",'1','2','3']].to_csv("/mnt/c/Users/44yos/RacePrediction/weekly_prediction/2021/dec_4/12_26_race_pred_v1.csv", index = False)

In [96]:
# summary[["pred",'1','2','3']].to_csv("/mnt/c/Users/44yos/RacePrediction/weekly_prediction/2021/dec_4/12_26_race_pred_v2.csv", index = False)

In [97]:
summary[["pred",'1','2','3']].to_csv("/mnt/c/Users/44yos/RacePrediction/weekly_prediction/2021/dec_4/12_26_race_pred_v3.csv", index = False)